In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/college/sem7/FYP/tweets_dataset - labelled.csv')
df.head()

,Unnamed: 0,user_name,user_location,user_description,user_verified,date,text,hashtags,source,label
0,0,Aravindh S,"Tamil Nadu, India",Nemophilist 🌎 90's KID 😉 😜,False,14-03-2022 12:38,And I am hearing new stories that within a sma...,NaN,Twitter for iPhone,0
1,1,Gbest Bulk SMS,Abuja Nigeria,Providers of Bulk SMS | Data Bundle | Airtime ...,False,14-03-2022 12:21,Do U have excess Airtime in ur line and will l...,NaN,Twitter for Android,neutral
2,2,Kalyanashis Mahanty,"Barabhum , West Bengal, India",School teacher by profession and I am everythi...,False,14-03-2022 12:05,@airtelindia @Airtel_Presence bye bye for now....,NaN,Twitter for Android,0
3,3,"Network Palava - Free, Cheap Data Daily.","Lagos, Nigeria","Get Cheap, Free Browsing Solutions at https://...",False,14-03-2022 11:59,Working NapsternetV Configuration Files Downlo...,['mtn'],Revive Social App,neutral
4,4,JayRoy 🇮🇳,NaN,Proud Indian \n\n(RT's r not endorsements),False,14-03-2022 11:55,@airtelnews @airtelindia @Airtel_Presence @air...,NaN,Twitter Web App,0


In [ ]:
df1 = df.drop(['user_name', 'user_location', 'user_description', 'user_verified', 'date', 'hashtags', 'source'], axis=1)
df1 = df1[df1['label'] != 'neutral']
df1 = df1.replace(np.nan, '', regex=True)

In [ ]:
df1.head()

,Unnamed: 0,text,label
0,0,And I am hearing new stories that within a sma...,0
2,2,@airtelindia @Airtel_Presence bye bye for now....,0
4,4,@airtelnews @airtelindia @Airtel_Presence @air...,0
5,5,@Airtel_Presence @WynkMusic @PawanKalyan @iams...,0
6,6,"Designed with ultimate sophistication, curated...",1


In [ ]:
df1['label'].value_counts()

0    267
1     39
       2
Name: label, dtype: int64

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
df1['text'] = df1['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df1['text'] = df1['text'].str.replace('[^\w\s]','')
stop = stopwords.words('english')
df1['text'] = df1['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])

In [ ]:
df1['text']= df1['text'].apply(space)

In [ ]:
df1.head()

,Unnamed: 0,text,label
0,0,hear new story within small 2bhk house nee use...,0
2,2,airtelindia airtel_presence bye bye bark two m...,0
4,4,airtelnews airtelindia airtel_presence airteli...,0
5,5,airtel_presence wynkmusic pawankalyan iamsamyu...,0
6,6,design ultimate sophistication curate love ele...,1


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(df1['text'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, df1['label'], test_size=0.25, random_state=5)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
MNB.fit(X_train, Y_train)

MultinomialNB()

In [ ]:
predicted = MNB.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(Y_test,predicted))
print(classification_report(Y_test,predicted))
print("Accuracy: ", accuracy_score(Y_test, predicted))

[[ 0  0  1]
 [ 0 65  3]
 [ 0  4  4]]
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.94      0.96      0.95        68
           1       0.50      0.50      0.50         8

    accuracy                           0.90        77
   macro avg       0.48      0.49      0.48        77
weighted avg       0.88      0.90      0.89        77

Accuracy:  0.8961038961038961


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.linear_model import SGDClassifier
SGDC = SGDClassifier()
SGDC.fit(X_train, Y_train)

SGDClassifier()

In [ ]:
predictions = SGDC.predict(X_test)

In [ ]:
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))
print("Accuracy: ", accuracy_score(Y_test, predictions))

[[ 0  1  0]
 [ 0 67  1]
 [ 0  6  2]]
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.91      0.99      0.94        68
           1       0.67      0.25      0.36         8

    accuracy                           0.90        77
   macro avg       0.52      0.41      0.44        77
weighted avg       0.87      0.90      0.87        77

Accuracy:  0.8961038961038961


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(criterion='entropy', random_state=1)
DT.fit(X_train, Y_train)

DecisionTreeClassifier(criterion='entropy', random_state=1)

In [ ]:
prediction = DT.predict(X_test)

In [ ]:
print(confusion_matrix(Y_test,prediction))
print(classification_report(Y_test,prediction))
print("Accuracy: ", accuracy_score(Y_test, prediction))

[[ 0  0  1]
 [ 0 66  2]
 [ 0  7  1]]
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.90      0.97      0.94        68
           1       0.25      0.12      0.17         8

    accuracy                           0.87        77
   macro avg       0.38      0.37      0.37        77
weighted avg       0.82      0.87      0.84        77

Accuracy:  0.8701298701298701


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.text, df1.label, test_size=0.2, random_state=32)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report
classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(train_vectors, y_train)
prediction_linear = classifier_linear.predict(test_vectors)

In [ ]:
print(classification_report(y_test,prediction_linear))

              precision    recall  f1-score   support

                   0.00      0.00      0.00         2
           0       0.88      0.96      0.92        53
           1       0.50      0.29      0.36         7

    accuracy                           0.85        62
   macro avg       0.46      0.42      0.43        62
weighted avg       0.81      0.85      0.83        62



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

CountVectorizer()

In [ ]:
X_train = vectorizer.transform(X_train)
X_test= vectorizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
predictions = classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print("Accuracy: ", accuracy_score(y_test, predictions))

[[ 0  2  0]
 [ 0 53  0]
 [ 0  7  0]]
              precision    recall  f1-score   support

                   0.00      0.00      0.00         2
           0       0.85      1.00      0.92        53
           1       0.00      0.00      0.00         7

    accuracy                           0.85        62
   macro avg       0.28      0.33      0.31        62
weighted avg       0.73      0.85      0.79        62

Accuracy:  0.8548387096774194


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
